In [1037]:
# This entrypoint file to be used in development. Start by reading README.md
import numpy as np
import random
from RPS_game import play, mrugesh, abbey, quincy, kris, human, random_player
from RPS import player
from unittest import main
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [1038]:
def player(prev_play, opponent_history=[], my_history=[]):
    def move_to_vector(move):
        mapping = {'R': [1,0,0],
                   'P': [0,1,0],
                   'S': [0,0,1],
                   '':  [1,0,0]}
        return mapping[move]

    if not hasattr(player, 'opponent_history'):
        player.opponent_history = []
        player.my_history = []

    # handle first move
    if prev_play == '':
        prev_play = 'R'
        guess = 'P' 
    else:
        guess = random.choice(['R', 'P', 'S'])

    player.opponent_history.append(prev_play)
    window_size = 100
    if len(player.opponent_history) > 20:
        #counter kris
        my_prev = player.my_history[-(window_size+1):-1]
        opp_now = player.opponent_history[-window_size:]
        def did_react(prev_my, current_opp):
            return ((prev_my == 'R' and current_opp == 'P') or
                    (prev_my == 'P' and current_opp == 'S') or
                    (prev_my == 'S' and current_opp == 'R'))

        reactive_matches = sum(did_react(m,o) for m,o in zip(my_prev, opp_now))
        frac = reactive_matches / len(my_prev)
        if frac > 0.9:
            guess = {'R':'S','P':'R','S':'P'}[player.my_history[-1]]
            player.my_history.append(guess)
            return guess
        #counter abbey strategy
        X = np.array([move_to_vector(m) for m in player.opponent_history[:-1]])
        y = np.array(player.opponent_history[1:])
        RPS = LogisticRegression(max_iter=1000, solver='lbfgs', multi_class='multinomial')
        RPS.fit(X, y)

        last_move_vec = move_to_vector(player.opponent_history[-1])
        probs = RPS.predict_proba([last_move_vec])[0]
        classes = RPS.classes_

        predicted_move = classes[np.argmax(probs)]
        counter = {'R': 'P', 'P': 'S', 'S': 'R'}
        guess = counter[predicted_move]
    player.my_history.append(guess)
    return guess

In [1039]:
play(player, quincy, 1000)
if hasattr(player, 'my_history'):
    del player.my_history
if hasattr(player, 'opponent_history'):
    del player.opponent_history
play(player, abbey, 1000)
if hasattr(player, 'my_history'):
    del player.my_history
if hasattr(player, 'opponent_history'):
    del player.opponent_history
play(player, kris, 1000)
if hasattr(player, 'my_history'):
    del player.my_history
if hasattr(player, 'opponent_history'):
    del player.opponent_history
play(player, mrugesh, 1000)

# play(human, abbey, 20, verbose=True)
# play(human, random_player, 1000)
# main(module='test_module', exit=False)





Final results: {'p1': 395, 'p2': 202, 'tie': 403}
Player 1 win rate: 66.1641541038526%
Final results: {'p1': 457, 'p2': 310, 'tie': 233}
Player 1 win rate: 59.58279009126467%
Final results: {'p1': 943, 'p2': 52, 'tie': 5}
Player 1 win rate: 94.77386934673366%
Final results: {'p1': 831, 'p2': 159, 'tie': 10}
Player 1 win rate: 83.93939393939394%


83.93939393939394

In [1022]:
%reset -f